In [1]:
%mkdir -p /content/drive/MyDrive/GSE261283
%cd /content/drive/MyDrive/GSE261283

/content/drive/MyDrive/GSE261283


In [ ]:
!wget "https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE261283&format=file&file=GSE261283%5Faggr%2Epost%5Fqc%2Eh5seurat" -O GSE261283_aggr.post.qc.h5seurat

--2024-07-27 15:21:56--  https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE261283&format=file&file=GSE261283%5Faggr%2Epost%5Fqc%2Eh5seurat
Resolving www.ncbi.nlm.nih.gov (www.ncbi.nlm.nih.gov)... 130.14.29.110, 2607:f220:41e:4290::110
Connecting to www.ncbi.nlm.nih.gov (www.ncbi.nlm.nih.gov)|130.14.29.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3855631162 (3.6G) [application/octet-stream]
Saving to: ‘GSE261283_aggr.post.qc.h5seurat’

GSE261283_aggr.post 100%[===================>]   3.59G  39.4MB/s    in 2m 11s  

2024-07-27 15:24:07 (28.0 MB/s) - ‘GSE261283_aggr.post.qc.h5seurat’ saved [3855631162/3855631162]



In [ ]:
!wget "https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE261283&format=file&file=GSE261283%5Fprotospacer%5Fcalls%5Fper%5Fcell%2Ecsv%2Egz" -O GSE261283_protospacer_calls_per_cell.csv.gz
!gzip -d GSE261283_protospacer_calls_per_cell.csv.gz

--2024-07-28 00:54:36--  https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE261283&format=file&file=GSE261283%5Fprotospacer%5Fcalls%5Fper%5Fcell%2Ecsv%2Egz
Resolving www.ncbi.nlm.nih.gov (www.ncbi.nlm.nih.gov)... 130.14.29.110, 2607:f220:41e:4290::110
Connecting to www.ncbi.nlm.nih.gov (www.ncbi.nlm.nih.gov)|130.14.29.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 648303 (633K) [application/octet-stream]
Saving to: ‘GSE261283_protospacer_calls_per_cell.csv.gz’

GSE261283_protospac 100%[===================>] 633.11K   595KB/s    in 1.1s    

2024-07-28 00:54:38 (595 KB/s) - ‘GSE261283_protospacer_calls_per_cell.csv.gz’ saved [648303/648303]



In [2]:
%pip install scanpy python-dotenv -q

import os
import tarfile
import json
import gzip

import scanpy as sc
import anndata as ad
import pandas as pd

from dotenv import load_dotenv
load_dotenv(dotenv_path='/content/drive/MyDrive/.gse.env')
COLS = json.loads(os.environ.get('OBS_COLS'))
ORGANISMS = json.loads(os.environ.get('ORGANISMS'))

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.4/124.4 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 5.0 MB/s eta 0:00:00


In [ ]:
%ls

GSE261283_aggr.post.qc.h5ad  GSE261283_aggr.post.qc.h5seurat  GSE261283.ipynb


In [ ]:
adata = sc.read_h5ad("./GSE261283_aggr.post.qc.h5ad")
adata

AnnData object with n_obs × n_vars = 40742 × 33538
    var: 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'

In [ ]:
import h5py

with h5py.File("./GSE261283_aggr.post.qc.h5seurat", "r") as f:
    print(list(f.keys()))
    barcodes = [x.decode() for x in f['cell.names']]
    print(barcodes[:10])
    obs = f['meta.data']
    data_dict = {'barcode': barcodes}
    for key in obs.keys():
        if isinstance(obs[key], h5py.Dataset):
            data_dict[key] = obs[key][:]
        else:
            data_dict[key] = obs[key]['codes']
    obs = pd.DataFrame(data_dict)
obs.head()

['active.ident', 'assays', 'cell.names', 'commands', 'graphs', 'images', 'meta.data', 'misc', 'neighbors', 'reductions', 'tools']
['AGTCACAAGAATCTAG-1', 'TCCACCATCAGACCTA-1', 'ACCAACACATCTCCCA-1', 'ACCAACATCTGGGCAC-1', 'TCCCATGAGAGCATAT-1', 'CGGAACCCAGAAACCG-1', 'CTAACTTAGGAACGAA-1', 'ACCATTTTCGGCCAAC-1', 'AAACCCATCCACAGGC-1', 'GAGTTTGCAGTGACCC-1']


,barcode,Pool,n_genes_by_counts,pct_counts_mt,total_counts,total_counts_mt
0,AGTCACAAGAATCTAG-1,0,10106,2.789805,81296,2268
1,TCCACCATCAGACCTA-1,0,8898,0.104335,74759,78
2,ACCAACACATCTCCCA-1,0,9303,0.853604,67010,572
3,ACCAACATCTGGGCAC-1,0,8509,0.051505,67955,35
4,TCCCATGAGAGCATAT-1,0,8630,1.360840,66797,909


In [ ]:
df = pd.read_csv("./GSE261283_protospacer_calls_per_cell.csv", index_col=0)
df.head()

,num_features,feature_call,num_umis
cell_barcode,,,
ACACTGAGTAGAATGT-1,1,RAB1A,8
AGCCACGGTGCTTATG-1,1,RAB1A,14
AGCTACAGTCTAGGCC-1,1,RAB1A,5
ATGAGTCCAAGTTCCA-1,1,RAB1A,3
ATTCTACTCAGCCCAG-1,1,RAB1A,9


In [ ]:
adata.obs = obs
adata.obs.set_index('barcode', inplace=True)
adata.obs.head()

,Pool,n_genes_by_counts,pct_counts_mt,total_counts,total_counts_mt
barcode,,,,,
AGTCACAAGAATCTAG-1,0,10106,2.789805,81296,2268
TCCACCATCAGACCTA-1,0,8898,0.104335,74759,78
ACCAACACATCTCCCA-1,0,9303,0.853604,67010,572
ACCAACATCTGGGCAC-1,0,8509,0.051505,67955,35
TCCCATGAGAGCATAT-1,0,8630,1.360840,66797,909


In [ ]:
obs_names =np.intersect1d(adata.obs_names, df.index)
adata = adata[obs_names]
df = df.loc[obs_names]

adata.obs[df.columns] = df.loc[adata.obs_names]
adata.obs.head()

<ipython-input-82-468750014a64>:5: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs[df.columns] = df.loc[adata.obs_names]


,Pool,n_genes_by_counts,pct_counts_mt,total_counts,total_counts_mt,num_features,feature_call,num_umis
barcode,,,,,,,,
AAACCCAAGAAGCGGG-1,0,5729,3.017132,19787,597,1,rs1919873_4,268
AAACCCAAGCCTGAAG-8,6,8513,5.231251,54270,2839,2,rs57246313_1|rs2332323_5,1041|4078
AAACCCAAGCTTACGT-8,6,7215,2.489260,40735,1014,1,rs10792352_2,368
AAACCCAAGGATTTGA-1,0,4444,6.514528,15074,982,1,rs4671647_5,560
AAACCCAAGGTACAAT-7,5,7309,2.860941,39113,1119,1,rs2332323_1,15


In [ ]:
!wget "https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE261283&format=file&file=GSE261283%5Ffeature%5Freference%2Ecsv%2Egz" -O GSE261283_featurereference.csv.gz
ref_df = pd.read_csv("./GSE261283_featurereference.csv.gz")
ref_df.head()

--2024-07-28 01:16:46--  https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE261283&format=file&file=GSE261283%5Ffeature%5Freference%2Ecsv%2Egz
Resolving www.ncbi.nlm.nih.gov (www.ncbi.nlm.nih.gov)... 130.14.29.110, 2607:f220:41e:4290::110
Connecting to www.ncbi.nlm.nih.gov (www.ncbi.nlm.nih.gov)|130.14.29.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5426 (5.3K) [application/octet-stream]
Saving to: ‘GSE261283_featurereference.csv.gz’

GSE261283_featurere 100%[===================>]   5.30K  --.-KB/s    in 0s      

2024-07-28 01:16:47 (653 MB/s) - ‘GSE261283_featurereference.csv.gz’ saved [5426/5426]



,id,name,feature_type,genome,read,pattern,sequence
0,rs113214136_3,rs113214136_3,CRISPR Guide Capture,NaN,R2,(BC)GTTTAAGAGCTAAGCTGGAA,GGGATTCGGAGCGAGAGACC
1,rs4722672-rs9770544_3,rs4722672-rs9770544_3,CRISPR Guide Capture,NaN,R2,(BC)GTTTAAGAGCTAAGCTGGAA,TGGCACCGGCAGGTCCCGGT
2,rs118115924_4,rs118115924_4,CRISPR Guide Capture,NaN,R2,(BC)GTTTAAGAGCTAAGCTGGAA,GGGGTGGTCTGTCTCATGCT
3,rs16908520_2,rs16908520_2,CRISPR Guide Capture,NaN,R2,(BC)GTTTAAGAGCTAAGCTGGAA,AGACAGAAGTATATAATTTG
4,rs883650_3,rs883650_3,CRISPR Guide Capture,NaN,R2,(BC)GTTTAAGAGCTAAGCTGGAA,GCACAGGCTATGCCTAGAAT


In [ ]:
adata.var

,mt,n_cells_by_counts,mean_counts,pct_dropout_by_counts,total_counts
ENSG00000243485,False,5,0.000115,99.988454,5
ENSG00000237613,False,1,0.000023,99.997691,1
ENSG00000186092,False,3,0.000069,99.993072,3
ENSG00000238009,False,150,0.003672,99.653620,159
ENSG00000239945,False,5,0.000115,99.988454,5
...,...,...,...,...,...
ENSG00000277856,False,0,0.000000,100.000000,0
ENSG00000275063,False,0,0.000000,100.000000,0
ENSG00000271254,False,6320,0.176793,85.405842,7656
ENSG00000277475,False,1,0.000023,99.997691,1


In [ ]:
gene_name_map = pd.read_excel("./media-2.xlsx", sheet_name="S2", header=1)
gene_name_map.head()

In [ ]:
gene_map = dict(zip(gene_name_map['sgRNA Name'].apply(lambda x: '-'.join(x.split(','))),
                    gene_name_map['Implicated Genes'].fillna('non-targeting')))
adata.obs['tmp'] = adata.obs['feature_call'].apply(
    lambda x: '+'.join([gene_map.get(y, 'Unknown') for y in x.split('|')])
)

In [ ]:
df.loc[adata.obs['tmp'] == 'Unknown']

,num_features,feature_call,num_umis
cell_barcode,,,


In [ ]:
df.loc[~adata.obs['tmp'].isna(), 'num_features'].unique()

array([1])

In [ ]:
adata.obs['tmp'].value_counts()

tmp
non-targeting                                         2145
AHNAK                                                  680
LGR4                                                   521
SHFM1                                                  514
ATRIP,NME6,PFKFB4,SHISA5,TREX1                         481
                                                      ... 
TSC22D2+CCDC170                                          1
PA2G4P4,SSR3,TIPARP+ATRIP,NME6,PFKFB4,SHISA5,TREX1       1
KARS+SSR3,TIPARP                                         1
SOCS2,SOCS2-AS1+JAG1,MKKS,SLX4IP+SOST                    1
AC113189.5,MPDU1+PSMG1,ETS2                              1
Name: count, Length: 3883, dtype: int64

In [ ]:
adata.obs['perturbation_name'] = adata.obs.pop('tmp')
adata.obs['perturbation_name'].value_counts()

perturbation_name
non-targeting                                         2145
AHNAK                                                  680
LGR4                                                   521
SHFM1                                                  514
ATRIP,NME6,PFKFB4,SHISA5,TREX1                         481
                                                      ... 
TSC22D2+CCDC170                                          1
PA2G4P4,SSR3,TIPARP+ATRIP,NME6,PFKFB4,SHISA5,TREX1       1
KARS+SSR3,TIPARP                                         1
SOCS2,SOCS2-AS1+JAG1,MKKS,SLX4IP+SOST                    1
AC113189.5,MPDU1+PSMG1,ETS2                              1
Name: count, Length: 3883, dtype: int64

In [ ]:
adata.obs['crispr_type'] = 'CRISPRi'
adata.obs['cancer_type'] = 'non-cancer'
adata.obs['cell_type'] = 'human fetal osteoblast 1.19 cells (hFOBs)'
adata.obs['organism'] = ORGANISMS[0]
adata.obs['condition'] = adata.obs['perturbation_name'].apply(
    lambda x: 'targeting' if x != 'non-targeting' else x
)

In [ ]:
missing = [col for col in COLS if col not in adata.obs]
print(f"Missing following columns: \n {missing}")

if 'organism' in adata.obs_keys():
    assert adata.obs['organism'].isin(ORGANISMS).all(), "Invalid organism naming"

Missing following columns: 
 []


In [3]:
%pip install mygene
import mygene

def ensembl_to_symbol(ensembl_ids):
    mg = mygene.MyGeneInfo()

    # Query mygene for the given Ensembl IDs
    results = mg.querymany(ensembl_ids, scopes='ensembl.gene', fields='symbol', species='human')

    # Create a dictionary to store the results
    id_to_symbol = {}

    for result in results:
        if 'symbol' in result:
            id_to_symbol[result['query']] = result['symbol']
        else:
            id_to_symbol[result['query']] = 'Not found'

    return id_to_symbol

In [9]:
gene_map = ensembl_to_symbol(adata.var_names)

INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:querying 1001-2000...
INFO:biothings.client:done.
INFO:biothings.client:querying 2001-3000...
INFO:biothings.client:done.
INFO:biothings.client:querying 3001-4000...
INFO:biothings.client:done.
INFO:biothings.client:querying 4001-5000...
INFO:biothings.client:done.
INFO:biothings.client:querying 5001-6000...
INFO:biothings.client:done.
INFO:biothings.client:querying 6001-7000...
INFO:biothings.client:done.
INFO:biothings.client:querying 7001-8000...
INFO:biothings.client:done.
INFO:biothings.client:querying 8001-9000...
INFO:biothings.client:done.
INFO:biothings.client:querying 9001-10000...
INFO:biothings.client:done.
INFO:biothings.client:querying 10001-11000...
INFO:biothings.client:done.
INFO:biothings.client:querying 11001-12000...
INFO:biothings.client:done.
INFO:biothings.client:querying 12001-13000...
INFO:biothings.client:done.
INFO:biothings.client:querying 13001-14000...
INFO:biothings

In [5]:
gene_map

{'ENSG00000243485': 'MIR1302-2HG',
 'ENSG00000237613': 'FAM138A',
 'ENSG00000186092': 'OR4F5',
 'ENSG00000238009': 'Not found',
 'ENSG00000239945': 'Not found',
 'ENSG00000239906': 'Not found',
 'ENSG00000241599': 'Not found',
 'ENSG00000236601': 'Not found',
 'ENSG00000284733': 'OR4F29',
 'ENSG00000235146': 'Not found',
 'ENSG00000284662': 'OR4F16',
 'ENSG00000229905': 'Not found',
 'ENSG00000237491': 'LINC01409',
 'ENSG00000177757': 'FAM87B',
 'ENSG00000225880': 'LINC00115',
 'ENSG00000230368': 'FAM41C',
 'ENSG00000272438': 'Not found',
 'ENSG00000230699': 'Not found',
 'ENSG00000241180': 'Not found',
 'ENSG00000223764': 'LINC02593',
 'ENSG00000187634': 'SAMD11',
 'ENSG00000188976': 'NOC2L',
 'ENSG00000187961': 'KLHL17',
 'ENSG00000187583': 'PLEKHN1',
 'ENSG00000187642': 'PERM1',
 'ENSG00000272512': 'Not found',
 'ENSG00000188290': 'HES4',
 'ENSG00000187608': 'ISG15',
 'ENSG00000224969': 'Not found',
 'ENSG00000188157': 'AGRN',
 'ENSG00000273443': 'Not found',
 'ENSG00000237330': 'RN

In [10]:
adata.var['gene_name'] = adata.var_names.map(gene_map)
adata.var.head()

,mt,n_cells_by_counts,mean_counts,pct_dropout_by_counts,total_counts,gene_name
ENSG00000243485,False,5,0.000115,99.988454,5,MIR1302-2HG
ENSG00000237613,False,1,0.000023,99.997691,1,FAM138A
ENSG00000186092,False,3,0.000069,99.993072,3,OR4F5
ENSG00000238009,False,150,0.003672,99.653620,159,Not found
ENSG00000239945,False,5,0.000115,99.988454,5,Not found


In [11]:
adata = adata[:, adata.var['gene_name'] != 'Not found']
adata.var.index.name = 'gene_id'
adata.var.reset_index(drop=False, inplace=True)
adata.var.set_index('gene_name', inplace=True)
adata.var.head()

,gene_id,mt,n_cells_by_counts,mean_counts,pct_dropout_by_counts,total_counts
gene_name,,,,,,
MIR1302-2HG,ENSG00000243485,False,5,0.000115,99.988454,5
FAM138A,ENSG00000237613,False,1,0.000023,99.997691,1
OR4F5,ENSG00000186092,False,3,0.000069,99.993072,3
OR4F29,ENSG00000284733,False,0,0.000000,100.000000,0
OR4F16,ENSG00000284662,False,0,0.000000,100.000000,0


In [12]:
adata.write_h5ad("./GSE261283_merged.h5ad")